In [26]:
#run this before running any subsequent below
#counting roughly how many pos/neg/neu are out there for combined Dartmouth set
import numpy as np 
import urllib.request 
import os 
import csv 
import requests 
import time
import math
import rando
import pandas as pd
from collections import defaultdict
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect
import heapq
import pickle
import warnings
import random
from sklearn.utils import shuffle

filepath2="C:\\Backup of covid project\\2cls_posVSnonpos\\data\\"
path="C:\\Backup of covid project\\2cls_posVSnonpos\\data\\"

types=["comment"]
years=['2019','2020']
#years=['2020']
schools = ["notredame","uofm","columbia","UCSD","berkeley","Harvard","ucla"]
ct_thr=150

#Positive: 1, negative:0
#2020:1, 2019:0
label_group=["label","unlabel"]
months=['08','09','10','11']

att_com=['time','id','body','Emotion','emo_pred']
drop_com=["link_id",'Topic',"parent_id","author_fullname","author","gildings","score","subreddit","no_follow","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
emo=["Very Positive","Positive","Neutral","Negative","Very negative"]
top=["Covid","Academics","Sports","Campus/Students Life","Social Media","Religion","Politics","Others"]

#Positive: 1, negative:0
num=[]
for t in types:
    for s in schools:
        num_sch=[]
        for y in years:
            pos=0
            neu=0
            neg=0
            for l in label_group:
                f=s+t+y+l+".csv"
                os.chdir(filepath2)
                data=pd.read_csv(f,skip_blank_lines=True)
                data=data[['body','Emotion']]
                ct=0
                for e in data['Emotion'].values:
                    row=[0,0]
                    if e=='Negative' or e=='Very negative':
                        neg+=1
                    elif e=='Positive' or e=='Very Positive':
                        pos+=1
                    elif e=='Neutral':
                        neu+=1
                    ct+=1
                    if ct>ct_thr:
                        break
            num_sch.append([neg,neu,pos])
        num.append(num_sch)
                
                
s='dartmouth'
f1='dartmouthcomment2019label.csv'
f2='dartmouthcomment2019unlabel.csv'
f3='dartmouthcomment2020label.csv'
f4='dartmouthcomment2020unlabel.csv'
os.chdir(filepath2)
data1=pd.read_csv(f1,skip_blank_lines=True)
data1=data1.drop(drop_com, axis=1)
data2=pd.read_csv(f2,skip_blank_lines=True)
data2=data2.drop(drop_com, axis=1)
data3=pd.read_csv(f3,skip_blank_lines=True)
data3=data3.drop(drop_com, axis=1)
data4=pd.read_csv(f4,skip_blank_lines=True)
data4=data4.drop(drop_com, axis=1)
data = pd.concat([data1,data2,data3,data4], axis=0).reset_index(drop=True)
data=data[['body','Emotion']]
pos=0
neu=0
neg=0
ct=0
for e in data['Emotion'].values:
    if e=='Negative' or e=='Very negative':
        neg+=1
    elif e=='Positive' or e=='Very Positive':
        pos+=1
    elif e=='Neutral':
        neu+=1
    ct+=1
    if ct>ct_thr:
        break
num_sch=[]
num_sch.append([0,0,0])
num_sch.append([neg,neu,pos])
num.append(num_sch)

prop=sum(sum(np.array(num)))
neu2neg_neu=prop[1]/(prop[0]+prop[1])
neu2pos_neu=prop[1]/(prop[2]+prop[1])
print(prop)

[549 861 574]


In [31]:
#Formualte train/test set by formulating sets of indices
#For pos VS non-pos
import numpy as np 
import urllib.request 
import os 
import csv 
import requests 
import time
import math
import rando
import pandas as pd
from collections import defaultdict
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect
import math
#stopword = stopwords.words(‘english’)
import warnings
import random
import scipy.sparse

path_result='C:\\Backup of covid project\\2cls_posVSnonpos\\results\\'
path_data="C:\\Backup of covid project\\2cls_posVSnonpos\\data\\"
path_code="C:\\Backup of covid project\\"


random.seed(10)
filepath=path_data
path=path_data
os.chdir(filepath)

types=["comment"]
years=['2019','2020']

label_group=["label","unlabel"]
months=['08','09','10','11']

drop_com=["link_id","parent_id","author_fullname","author","gildings","score","subreddit","no_follow","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
att_com=["time","id","body","Emotion","Topic"]

#emo=["Very Positive","Positive","Neutral","Negative","Very negative"]
emo=["Very negative","Negative","Neutral","Positive","Very Positive"]
top=["Covid","Academics","Sports","Campus/Students Life","Social Media","Religion","Politics","Others"]




#FIRST DEAL WITH DARTMOUTH DATA
#schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
schools = ["dartmouth"]
for t in types:
    for s in schools:
        train_index=[]
        test_index=[]
        to_be_labeled_index=[]
        idx=0
        n_neu=np.round(50*neu2neg_neu)
        n_test=[50-n_neu,n_neu,50]
        for y in years:
            for l in label_group:
                f=s+t+y+l+".csv"
                os.chdir(filepath)
                data=pd.read_csv(f,skip_blank_lines=True)
                data=data.drop(drop_com, axis=1)
                for d in range(len(data['id'])):
                    if str(data['time'][d])[4:6] in months:
                        if isinstance(data['Emotion'][d],str)==False:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[0] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                        elif data['Emotion'][d]==emo[0]:
                            train_index.append(idx)
                        elif data['Emotion'][d]==emo[1] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                        elif data['Emotion'][d]==emo[1]:
                            train_index.append(idx)
                        elif data['Emotion'][d]==emo[2] and n_test[1]>0:
                            test_index.append(idx)
                            n_test[1]=n_test[1]-1
                        elif data['Emotion'][d]==emo[2]:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[3] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                        elif data['Emotion'][d]==emo[3]:
                            train_index.append(idx)
                        elif data['Emotion'][d]==emo[4] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                        elif data['Emotion'][d]==emo[4]:
                            train_index.append(idx)
                        else:
                            to_be_labeled_index.append(idx)
            
                        os.chdir(path)  
                        
            
                        idx=idx+1
                        if idx/600-int(idx/600)==0:
                            print(s+str(idx))    
        os.chdir(path)
        name="train_index_"+s+"_cm.p"
        pickle.dump(train_index,open(name,"wb"))
        name="test_index_"+s+"_cm.p"
        pickle.dump(test_index,open(name,"wb"))
        name="to_be_labeled_index_"+s+"_cm.p"
        pickle.dump(to_be_labeled_index,open(name,"wb"))
        
        
        
        
#Deal with other school data
schools = ["notredame","uofm","columbia","UCSD","berkeley","Harvard","ucla"]
#schools = ["dartmouth"]
for t in types:
    for s in schools:
        for y in years:
            train_index=[]
            test_index=[]
            to_be_labeled_index=[]
            idx=0
            n_neu=np.round(50*neu2neg_neu)
            n_test=[50-n_neu,n_neu,50]
            for l in label_group:
                f=s+t+y+l+".csv"
                os.chdir(filepath)
                data=pd.read_csv(f,skip_blank_lines=True)
                data=data.drop(drop_com, axis=1)
                for d in range(len(data['id'])):
                    if str(data['time'][d])[4:6] in months:
                        if isinstance(data['Emotion'][d],str)==False:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[0] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                        elif data['Emotion'][d]==emo[0]:
                            train_index.append(idx)
                        elif data['Emotion'][d]==emo[1] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                        elif data['Emotion'][d]==emo[1]:
                            train_index.append(idx)
                        elif data['Emotion'][d]==emo[2] and n_test[1]>0:
                            test_index.append(idx)
                            n_test[1]=n_test[1]-1
                        elif data['Emotion'][d]==emo[2]:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[3] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                        elif data['Emotion'][d]==emo[3]:
                            train_index.append(idx)
                        elif data['Emotion'][d]==emo[4] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                        elif data['Emotion'][d]==emo[4]:
                            train_index.append(idx)
                        else:
                            to_be_labeled_index.append(idx)
            
                        os.chdir(path)  
                        
            
                        idx=idx+1
                        if idx/600-int(idx/600)==0:
                            print(s+str(idx))    
            os.chdir(path)
            name="train_index_"+s+y+"_cm.p"
            pickle.dump(train_index,open(name,"wb"))
            name="test_index_"+s+y+"_cm.p"
            pickle.dump(test_index,open(name,"wb"))
            name="to_be_labeled_index_"+s+y+"_cm.p"
            pickle.dump(to_be_labeled_index,open(name,"wb"))

dartmouth600
dartmouth1200
dartmouth1800
notredame600
notredame1200
notredame600
notredame1200
notredame1800
notredame2400
notredame3000
notredame3600
notredame4200
notredame4800
notredame5400
notredame6000
notredame6600
uofm600
uofm1200
uofm1800
uofm2400
uofm3000
uofm3600
uofm4200
uofm4800
uofm5400
uofm6000
uofm6600
uofm7200
uofm7800
uofm8400
uofm9000
uofm9600
uofm10200
uofm600
uofm1200
uofm1800
uofm2400
uofm3000
uofm3600
uofm4200
uofm4800
uofm5400
uofm6000
uofm6600
uofm7200
uofm7800
uofm8400
uofm9000
uofm9600
uofm10200
uofm10800
uofm11400
columbia600
columbia1200
columbia1800
columbia2400
columbia3000
columbia3600
columbia4200
columbia600
columbia1200
columbia1800
columbia2400
columbia3000
columbia3600
columbia4200
columbia4800
columbia5400
columbia6000
columbia6600
columbia7200
UCSD600
UCSD1200
UCSD1800
UCSD2400
UCSD3000
UCSD3600
UCSD4200
UCSD4800
UCSD5400
UCSD6000
UCSD6600
UCSD7200
UCSD7800
UCSD8400
UCSD9000
UCSD9600
UCSD10200
UCSD10800
UCSD11400
UCSD12000
UCSD600
UCSD1200
UCSD1800

In [28]:
#counting how much training sample we have for DM set 
#and count ho much testing sample we have for each dataset
#for pos VS non-pos
import numpy as np 
import urllib.request 
import os 
import csv 
import requests 
import time
import math
import rando
import pandas as pd
from collections import defaultdict
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect

import math
#stopword = stopwords.words(‘english’)
import warnings
import random
import scipy.sparse

path_result='C:\\Backup of covid project\\2cls_posVSnonpos\\results\\'
path_data="C:\\Backup of covid project\\2cls_posVSnonpos\\data\\"
path_code="C:\\Backup of covid project\\"

random.seed(10)
filepath=path_data
path=path_data
os.chdir(filepath)

types=["comment"]
years=['2019','2020']

label_group=["label","unlabel"]
months=['08','09','10','11']

drop_com=["link_id","parent_id","author_fullname","author","gildings","score","subreddit","no_follow","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
att_com=["time","id","body","Emotion","Topic"]

#emo=["Very Positive","Positive","Neutral","Negative","Very negative"]
emo=["Very negative","Negative","Neutral","Positive","Very Positive"]
top=["Covid","Academics","Sports","Campus/Students Life","Social Media","Religion","Politics","Others"]




#FIRST DEAL WITH DARTMOUTH DATA
#schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
schools = ["dartmouth"]
for t in types:
    for s in schools:
        train_index=[]
        test_index=[]
        to_be_labeled_index=[]
        idx=0
        n_neu=np.round(50*neu2neg_neu)
        n_test=[50-n_neu,n_neu,50]
        train_neg=0
        train_pos=0
        train_neu=0
        test_neg=0
        test_pos=0
        test_neu=0
        for y in years:
            for l in label_group:
                f=s+t+y+l+".csv"
                os.chdir(filepath)
                data=pd.read_csv(f,skip_blank_lines=True)
                data=data.drop(drop_com, axis=1)
                for d in range(len(data['id'])):
                    if str(data['time'][d])[4:6] in months:
                        if isinstance(data['Emotion'][d],str)==False:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[0] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                            test_neg+=1
                        elif data['Emotion'][d]==emo[0]:
                            train_index.append(idx)
                            train_neg+=1
                        elif data['Emotion'][d]==emo[1] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                            test_neg+=1
                        elif data['Emotion'][d]==emo[1]:
                            train_index.append(idx)
                            train_neg+=1
                        elif data['Emotion'][d]==emo[2] and n_test[1]>0:
                            test_index.append(idx)
                            n_test[1]=n_test[1]-1
                            test_neu+=1
                        elif data['Emotion'][d]==emo[2]:
                            train_index.append(idx)
                            train_neu+=1
                        elif data['Emotion'][d]==emo[3] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                            test_pos+=1
                        elif data['Emotion'][d]==emo[3]:
                            train_index.append(idx)
                            train_pos+=1
                        elif data['Emotion'][d]==emo[4] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                            test_pos+=1
                        elif data['Emotion'][d]==emo[4]:
                            train_index.append(idx)
                            train_pos+=1
                        else:
                            to_be_labeled_index.append(idx)
            
                        os.chdir(path)  
                        
            
                        idx=idx+1
                        #if idx/600-int(idx/600)==0:
                        #    print(s+str(idx))    
        print('training numbers in DM: '+str([train_neg,train_neu,train_pos]))
        print('testing numbers in DM: '+str([test_neg,test_neu,test_pos]))
        
        
#Deal with other schools
schools = ["notredame","uofm","columbia","UCSD","berkeley","Harvard","ucla"]
#schools = ["dartmouth"]
for t in types:
    for s in schools:
        for y in years:
            train_index=[]
            test_index=[]
            to_be_labeled_index=[]
            idx=0
            n_neu=np.round(50*neu2neg_neu)
            n_test=[50-n_neu,n_neu,50]
            test_neg=0
            test_pos=0
            test_neu=0
            for l in label_group:
                f=s+t+y+l+".csv"
                os.chdir(filepath)
                data=pd.read_csv(f,skip_blank_lines=True)
                data=data.drop(drop_com, axis=1)
                for d in range(len(data['id'])):
                    if str(data['time'][d])[4:6] in months:
                        if isinstance(data['Emotion'][d],str)==False:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[0] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                            test_neg+=1
                        elif data['Emotion'][d]==emo[0]:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[1] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                            test_neg+=1
                        elif data['Emotion'][d]==emo[1]:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[2] and n_test[1]>0:
                            test_index.append(idx)
                            n_test[1]=n_test[1]-1
                            test_neu+=1
                        elif data['Emotion'][d]==emo[2]:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[3] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                            test_pos+=1
                        elif data['Emotion'][d]==emo[3]:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[4] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                            test_pos+=1
                        elif data['Emotion'][d]==emo[4]:
                            to_be_labeled_index.append(idx)
                        else:
                            to_be_labeled_index.append(idx)
            
                        os.chdir(path)  
                        
            
                        idx=idx+1
                        #if idx/600-int(idx/600)==0:
                        #    print(s+str(idx))    
            print('testing numbers in '+s+y+": "+str([test_neg,test_neu,test_pos]))

training numbers in DM: [125, 277, 293]
testing numbers in DM: [19, 31, 50]
testing numbers in notredame2019: [19, 31, 50]
testing numbers in notredame2020: [19, 31, 50]
testing numbers in uofm2019: [19, 31, 50]
testing numbers in uofm2020: [19, 31, 50]
testing numbers in columbia2019: [19, 31, 50]
testing numbers in columbia2020: [19, 31, 50]
testing numbers in UCSD2019: [19, 31, 50]
testing numbers in UCSD2020: [19, 31, 50]
testing numbers in berkeley2019: [19, 31, 50]
testing numbers in berkeley2020: [19, 31, 50]
testing numbers in Harvard2019: [19, 31, 50]
testing numbers in Harvard2020: [19, 31, 50]
testing numbers in ucla2019: [19, 31, 50]
testing numbers in ucla2020: [19, 31, 50]


In [32]:
#Formualte train/test set by formulating sets of indices
#For neg VS non-neg
import numpy as np 
import urllib.request 
import os 
import csv 
import requests 
import time
import math
import rando
import pandas as pd
from collections import defaultdict
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect

import math
#stopword = stopwords.words(‘english’)
import warnings
import random
import scipy.sparse

path_result='C:\\Backup of covid project\\2cls_negVSnonneg\\results\\'
path_data="C:\\Backup of covid project\\2cls_negVSnonneg\\data\\"
path_code="C:\\Backup of covid project\\"

random.seed(10)
filepath=path_data
path=path_data
os.chdir(filepath)

types=["comment"]
years=['2019','2020']

label_group=["label","unlabel"]
months=['08','09','10','11']

drop_com=["link_id","parent_id","author_fullname","author","gildings","score","subreddit","no_follow","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
att_com=["time","id","body","Emotion","Topic"]

#emo=["Very Positive","Positive","Neutral","Negative","Very negative"]
emo=["Very negative","Negative","Neutral","Positive","Very Positive"]
top=["Covid","Academics","Sports","Campus/Students Life","Social Media","Religion","Politics","Others"]




#FIRST DEAL WITH DARTMOUTH DATA
#schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
schools = ["dartmouth"]
for t in types:
    for s in schools:
        train_index=[]
        test_index=[]
        to_be_labeled_index=[]
        idx=0
        #n_neu=np.round(50*neu2neg_neu)
        #n_test=[50-n_neu,n_neu,50]
        
        n_neu=np.round(50*neu2pos_neu)
        n_test=[50,n_neu,50-n_neu]
        for y in years:
            for l in label_group:
                f=s+t+y+l+".csv"
                os.chdir(filepath)
                data=pd.read_csv(f,skip_blank_lines=True)
                data=data.drop(drop_com, axis=1)
                for d in range(len(data['id'])):
                    if str(data['time'][d])[4:6] in months:
                        if isinstance(data['Emotion'][d],str)==False:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[0] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                        elif data['Emotion'][d]==emo[0]:
                            train_index.append(idx)
                        elif data['Emotion'][d]==emo[1] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                        elif data['Emotion'][d]==emo[1]:
                            train_index.append(idx)
                        elif data['Emotion'][d]==emo[2] and n_test[1]>0:
                            test_index.append(idx)
                            n_test[1]=n_test[1]-1
                        elif data['Emotion'][d]==emo[2]:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[3] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                        elif data['Emotion'][d]==emo[3]:
                            train_index.append(idx)
                        elif data['Emotion'][d]==emo[4] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                        elif data['Emotion'][d]==emo[4]:
                            train_index.append(idx)
                        else:
                            to_be_labeled_index.append(idx)
            
                        os.chdir(path)  
                        
            
                        idx=idx+1
                        if idx/600-int(idx/600)==0:
                            print(s+str(idx))    
        os.chdir(path)
        name="train_index_"+s+"_cm.p"
        pickle.dump(train_index,open(name,"wb"))
        name="test_index_"+s+"_cm.p"
        pickle.dump(test_index,open(name,"wb"))
        name="to_be_labeled_index_"+s+"_cm.p"
        pickle.dump(to_be_labeled_index,open(name,"wb"))
        
        
        
        
#deal with other schools
schools = ["notredame","uofm","columbia","UCSD","berkeley","Harvard","ucla"]
#schools = ["dartmouth"]
for t in types:
    for s in schools:
        for y in years:
            train_index=[]
            test_index=[]
            to_be_labeled_index=[]
            idx=0
            n_neu=np.round(50*neu2pos_neu)
            n_test=[50,n_neu,50-n_neu]
            for l in label_group:
                f=s+t+y+l+".csv"
                os.chdir(filepath)
                data=pd.read_csv(f,skip_blank_lines=True)
                data=data.drop(drop_com, axis=1)
                for d in range(len(data['id'])):
                    if str(data['time'][d])[4:6] in months:
                        if isinstance(data['Emotion'][d],str)==False:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[0] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                        elif data['Emotion'][d]==emo[0]:
                            train_index.append(idx)
                        elif data['Emotion'][d]==emo[1] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                        elif data['Emotion'][d]==emo[1]:
                            train_index.append(idx)
                        elif data['Emotion'][d]==emo[2] and n_test[1]>0:
                            test_index.append(idx)
                            n_test[1]=n_test[1]-1
                        elif data['Emotion'][d]==emo[2]:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[3] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                        elif data['Emotion'][d]==emo[3]:
                            train_index.append(idx)
                        elif data['Emotion'][d]==emo[4] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                        elif data['Emotion'][d]==emo[4]:
                            train_index.append(idx)
                        else:
                            to_be_labeled_index.append(idx)
            
                        os.chdir(path)  
                        
            
                        idx=idx+1
                        if idx/600-int(idx/600)==0:
                            print(s+str(idx))    
            os.chdir(path)
            name="train_index_"+s+y+"_cm.p"
            pickle.dump(train_index,open(name,"wb"))
            name="test_index_"+s+y+"_cm.p"
            pickle.dump(test_index,open(name,"wb"))
            name="to_be_labeled_index_"+s+y+"_cm.p"
            pickle.dump(to_be_labeled_index,open(name,"wb"))

dartmouth600
dartmouth1200
dartmouth1800
notredame600
notredame1200
notredame600
notredame1200
notredame1800
notredame2400
notredame3000
notredame3600
notredame4200
notredame4800
notredame5400
notredame6000
notredame6600
uofm600
uofm1200
uofm1800
uofm2400
uofm3000
uofm3600
uofm4200
uofm4800
uofm5400
uofm6000
uofm6600
uofm7200
uofm7800
uofm8400
uofm9000
uofm9600
uofm10200
uofm600
uofm1200
uofm1800
uofm2400
uofm3000
uofm3600
uofm4200
uofm4800
uofm5400
uofm6000
uofm6600
uofm7200
uofm7800
uofm8400
uofm9000
uofm9600
uofm10200
uofm10800
uofm11400
columbia600
columbia1200
columbia1800
columbia2400
columbia3000
columbia3600
columbia4200
columbia600
columbia1200
columbia1800
columbia2400
columbia3000
columbia3600
columbia4200
columbia4800
columbia5400
columbia6000
columbia6600
columbia7200
UCSD600
UCSD1200
UCSD1800
UCSD2400
UCSD3000
UCSD3600
UCSD4200
UCSD4800
UCSD5400
UCSD6000
UCSD6600
UCSD7200
UCSD7800
UCSD8400
UCSD9000
UCSD9600
UCSD10200
UCSD10800
UCSD11400
UCSD12000
UCSD600
UCSD1200
UCSD1800

In [25]:
#counting how much training sample we have for DM set 
#and count ho much testing sample we have for each dataset
#for neg VS non-neg
import numpy as np 
import urllib.request 
import os 
import csv 
import requests 
import time
import math
import rando
import pandas as pd
from collections import defaultdict
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect
import math
#stopword = stopwords.words(‘english’)
import warnings
import random
import scipy.sparse

path_result='C:\\Backup of covid project\\2cls_negVSnonneg\\results\\'
path_data="C:\\Backup of covid project\\2cls_negVSnonneg\\data\\"
path_code="C:\\Backup of covid project\\"

random.seed(10)
filepath=path_data
path=path_data
os.chdir(filepath)

types=["comment"]
years=['2019','2020']

label_group=["label","unlabel"]
months=['08','09','10','11']

drop_com=["link_id","parent_id","author_fullname","author","gildings","score","subreddit","no_follow","total_awards_received","all_awardings","is_submitter","locked","send_replies","stickied",]
att_com=["time","id","body","Emotion","Topic"]

#emo=["Very Positive","Positive","Neutral","Negative","Very negative"]
emo=["Very negative","Negative","Neutral","Positive","Very Positive"]
top=["Covid","Academics","Sports","Campus/Students Life","Social Media","Religion","Politics","Others"]




#FIRST DEAL WITH DARTMOUTH DATA
#schools = ["notredame","uofm","columbia","dartmouth","UCSD","berkeley","Harvard","ucla"]
schools = ["dartmouth"]
for t in types:
    for s in schools:
        train_index=[]
        test_index=[]
        to_be_labeled_index=[]
        idx=0
        n_neu=np.round(50*neu2pos_neu)
        n_test=[50,n_neu,50-n_neu]
        train_neg=0
        train_pos=0
        train_neu=0
        test_neg=0
        test_pos=0
        test_neu=0
        for y in years:
            for l in label_group:
                f=s+t+y+l+".csv"
                os.chdir(filepath)
                data=pd.read_csv(f,skip_blank_lines=True)
                data=data.drop(drop_com, axis=1)
                for d in range(len(data['id'])):
                    if str(data['time'][d])[4:6] in months:
                        if isinstance(data['Emotion'][d],str)==False:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[0] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                            test_neg+=1
                        elif data['Emotion'][d]==emo[0]:
                            train_index.append(idx)
                            train_neg+=1
                        elif data['Emotion'][d]==emo[1] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                            test_neg+=1
                        elif data['Emotion'][d]==emo[1]:
                            train_index.append(idx)
                            train_neg+=1
                        elif data['Emotion'][d]==emo[2] and n_test[1]>0:
                            test_index.append(idx)
                            n_test[1]=n_test[1]-1
                            test_neu+=1
                        elif data['Emotion'][d]==emo[2]:
                            train_index.append(idx)
                            train_neu+=1
                        elif data['Emotion'][d]==emo[3] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                            test_pos+=1
                        elif data['Emotion'][d]==emo[3]:
                            train_index.append(idx)
                            train_pos+=1
                        elif data['Emotion'][d]==emo[4] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                            test_pos+=1
                        elif data['Emotion'][d]==emo[4]:
                            train_index.append(idx)
                            train_pos+=1
                        else:
                            to_be_labeled_index.append(idx)
            
                        os.chdir(path)  
                        
            
                        idx=idx+1
                        #if idx/600-int(idx/600)==0:
                        #    print(s+str(idx))    
        print('training numbers in DM: '+str([train_neg,train_neu,train_pos]))
        print('testing numbers in DM: '+str([test_neg,test_neu,test_pos]))
        
#Deal with other schools
schools = ["notredame","uofm","columbia","UCSD","berkeley","Harvard","ucla"]
#schools = ["dartmouth"]
for t in types:
    for s in schools:
        for y in years:
            train_index=[]
            test_index=[]
            to_be_labeled_index=[]
            idx=0
            n_neu=np.round(50*neu2pos_neu)
            n_test=[50,n_neu,50-n_neu]
            test_neg=0
            test_pos=0
            test_neu=0
            for l in label_group:
                f=s+t+y+l+".csv"
                os.chdir(filepath)
                data=pd.read_csv(f,skip_blank_lines=True)
                data=data.drop(drop_com, axis=1)
                for d in range(len(data['id'])):
                    if str(data['time'][d])[4:6] in months:
                        if isinstance(data['Emotion'][d],str)==False:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[0] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                            test_neg+=1
                        elif data['Emotion'][d]==emo[0]:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[1] and n_test[0]>0:
                            test_index.append(idx)
                            n_test[0]=n_test[0]-1
                            test_neg+=1
                        elif data['Emotion'][d]==emo[1]:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[2] and n_test[1]>0:
                            test_index.append(idx)
                            n_test[1]=n_test[1]-1
                            test_neu+=1
                        elif data['Emotion'][d]==emo[2]:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[3] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                            test_pos+=1
                        elif data['Emotion'][d]==emo[3]:
                            to_be_labeled_index.append(idx)
                        elif data['Emotion'][d]==emo[4] and n_test[2]>0:
                            test_index.append(idx)
                            n_test[2]=n_test[2]-1
                            test_pos+=1
                        elif data['Emotion'][d]==emo[4]:
                            to_be_labeled_index.append(idx)
                        else:
                            to_be_labeled_index.append(idx)
            
                        os.chdir(path)  
                        
            
                        idx=idx+1
                        #if idx/600-int(idx/600)==0:
                        #    print(s+str(idx))    
            print('testing numbers in '+s+y+": "+str([test_neg,test_neu,test_pos]))

training numbers in DM: [94, 277, 324]
testing numbers in DM: [50, 31, 19]
testing numbers in notredame2019: [50, 31, 19]
testing numbers in notredame2020: [50, 31, 19]
testing numbers in uofm2019: [50, 31, 19]
testing numbers in uofm2020: [50, 31, 19]
testing numbers in columbia2019: [50, 31, 19]
testing numbers in columbia2020: [50, 31, 19]
testing numbers in UCSD2019: [50, 31, 19]
testing numbers in UCSD2020: [50, 31, 19]
testing numbers in berkeley2019: [50, 31, 19]
testing numbers in berkeley2020: [50, 31, 19]
testing numbers in Harvard2019: [50, 31, 19]
testing numbers in Harvard2020: [50, 31, 19]
testing numbers in ucla2019: [50, 31, 19]
testing numbers in ucla2020: [50, 31, 19]
